In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.functional import interpolate
import numpy as np
import numpy.random as npr
from tqdm import trange
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
# function to reshape images
def change_dataset(changed, dataset, BS):
    ind = min(len(dataset),BS)
    X = np.zeros((ind,300,300,3))
    
    if ind != len(dataset):
        X = dataset[:ind]
        X = X.reshape((ind,300,300,3))
        res = np.zeros((X.shape[0],3,300,300))
        for i in range(X.shape[0]):
            res[i] = np.moveaxis(X[i],-1,0)
        res = interpolate(torch.tensor(res),size=(175,175),mode='bicubic',align_corners=False)
        
        if len(changed) == 1:
            changed = np.vstack([changed,np.array(res)])
            changed = changed [1:]
        else:
            changed = np.vstack([changed,np.array(res)])
        dataset = dataset[ind:]
        
    else:
        X = dataset
        X = X.reshape((ind,300,300,3))
        res = np.zeros((X.shape[0],3,300,300))
        for i in range(X.shape[0]):
            res[i] = np.moveaxis(X[i],-1,0)
        res = interpolate(torch.tensor(res),size=(175,175),mode='bicubic',align_corners=False)
        changed = np.vstack([changed,np.array(res)])
        dataset = np.array([])
    return changed, dataset

def min_max_scale(data):
    for i in range(len(data)):
        data[i] = (data[i]-np.min(data[i]))/(np.max(data[i])-np.min(data[i]))
    return data


# define training hyperparameters
BS = 128
EPOCHS = 100

In [3]:
# ----------- PREPARING DATA ------------- #
# loading Data
dataset = np.load('../share/data_train.npy')
t_train = np.load('../share/labels_train.npy')
print(f'dataset.shape, labels.shape: {dataset.shape, t_train.shape}')


# fixing mislabeled data
location = [880, 165, 558, 2396, 3198, 3715, 3734, 3824, 4412, 5238, 38, 2303, 3127, 3467, 3515, 5404, 6178, 248, 827, 1139, 1799, 2491, 2546, 4533, 6024, 103, 392, 846, 913, 1487, 2971, 3234, 3851, 4008, 4405, 4442, 4952, 5348, 204, 272, 297, 1177, 1198, 1326, 1655, 2156, 2690, 2864, 3450, 5095, 5111, 5186, 5243, 5785, 6106,387, 3973, 5112, 5331, 5002, 5263, 6046, 1843, 2941, 3419, 4158, 4166, 4369, 4980, 689, 973, 3066, 3028, 4905, 5107, 5708, 5817, 357, 368, 1065, 2093, 3229, 3341, 4469, 5257, 5711]
true_label = [6, 4, 7, 9, 5, 6, 3, 8, 2, 0, 5, 1, 4, 6, 3, 7, 9, 2, 7, 2, 8, 5, 1, 0, 6, 5, 2, 5, 6, 0, 5, 5, 5, 8, 5, 2, 5, 1, 4, 4, 4, 1, 4, 7, 0, 4, 4, 4, 9, 4, 4, 4, 8, 4, 0, 8, 3, 5, 2, 5, 0, 4, 6, 8, 0, 6, 9, 5, 2, 0, 3, 6, 7, 2, 5, 7, 1, 2, 7, 0, 5, 8,3, 1, 6, 8]

for k in range(len(location)):
    t_train[location[k]] = true_label[k]

# downsample data
dataset = dataset.T
X_train = np.zeros((1,3,175,175), dtype = np.float32)
while len(dataset) != 0:
    X_train, dataset = change_dataset(X_train,dataset,64)
print(f'2. dataset.shape, labels.shape: {X_train.shape, t_train.shape}')

# min-max scaling
X_train = X_train.reshape((len(X_train), 3*175*175))
mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_train = X_train.reshape((len(X_train),3,175,175))
print(f'3. X_train.shape, labels.shape: {X_train.shape, t_train.shape}')

dataset.shape, labels.shape: ((270000, 6195), (6195,))
2. dataset.shape, labels.shape: ((6195, 3, 175, 175), (6195,))
3. X_train.shape, labels.shape: ((6195, 3, 175, 175), (6195,))


In [4]:
# define cnn class with variable inputs
class cnn(nn.Module):
    def __init__(self, numChannels, classes, ks):
        # call the parent constructor
        super(cnn, self).__init__()
        # initialize first set of CONV => POOL => RELU layers
        self.conv1 = nn.Conv2d(in_channels=numChannels, out_channels=200,
            kernel_size=ks, stride=2, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3)
        self.relu1 = nn.ReLU(inplace=True)
        
        # initialize second set of CONV => POOL => RELU layers
        self.dropout1 = nn.Dropout(0.6)
        self.conv2 = nn.Conv2d(in_channels=200, out_channels=650,
            kernel_size=ks, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3)
        self.relu2 = nn.ReLU(inplace=True)
                               
        # initialize final set of CONV => POOL => RELU layers
        self.dropout2 = nn.Dropout(0.6)
        self.conv3 = nn.Conv2d(in_channels=650, out_channels=350,
            kernel_size=ks, padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3)
        self.relu3 = nn.ReLU(inplace=True)
                               
        # initialize our softmax classifier
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features=(3150 if ks[0]==3 else 1400), out_features=128)
        self.relu4 = nn.ReLU(inplace=True)
            
        self.dropout4 = nn.Dropout(0.6)
        self.fc3 = nn.Linear(in_features=128, out_features=classes)
        self.logSoftmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        # pass the input through our first set of CONV => RELU =>
        # POOL layers
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        # pass the output from the previous layer through the second
        # set of CONV => RELU => POOL layers
        x = self.dropout1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x) 
        
        # through final CONV => RELU => POOL layer
        x = self.dropout2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        
        # pass the output to our softmax classifier to get our output
        # predictions
        m = nn.Flatten()
        x = m(x)
        x = self.dropout3(x)
        x = self.fc2(x)
        x = self.relu4(x)
        x = self.dropout4(x)
        x = self.fc3(x)
        output = self.logSoftmax(x)
        # return the output predictions
        return output

In [5]:
# accuracy function
def accuracy(pred, y):
    _, predictions = pred.max(1)
    correct = (predictions == y).sum()
    samples = predictions.size(0)
    return correct/samples

# training function
def train(model, optim, lossFn, loader_train, loader_val, epochs=50, device='cpu'):
    model.to(device)
    
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []

    for e in (t := trange(epochs)):

        # train
        epochloss=0; epochacc=0

        model.train()

        for _, (x,y) in enumerate(loader_train):
            x = x.to(device)
            y = y.to(device)

            # perform a forward pass and calculate the training loss
            # zero out the gradients
            optim.zero_grad()
            pred = model(x)
            loss = lossFn(pred,y)
            # backprop
            loss.backward()
            # find acc
            acc = accuracy(pred, y)
            # update weights
            optim.step()
            epochloss += loss.item()
            epochacc += acc.item()
        train_loss = epochloss/len(loader_train)
        train_acc = epochacc/len(loader_train)

        # evaluate
        epochloss=0; epochacc=0

        model.eval()

        with torch.no_grad():
            for _, (x,y) in enumerate(loader_val):
                x = x.to(device)
                y = y.to(device)
                # make the predictions
                pred = model(x)
                loss = lossFn(pred,y)
                # find acc
                acc = accuracy(pred, y)
                epochloss += loss.item()
                epochacc += acc.item()
        val_loss = epochloss/len(loader_val)
        val_acc = epochacc/len(loader_val)

        # add to lists
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        t.set_description('loss %.2f acc %.3f' % (val_loss, val_acc))
    
    model.to(torch.device('cpu'))
    return train_losses, train_accs, val_losses, val_accs


In [6]:
# 4-Fold Cross Validation
from sklearn.model_selection import KFold
k = 4 # number of folds
kf = KFold(n_splits=k,shuffle=True)
# split training feature matrix into training and validation sets
f=1
for train_index, validation_index in kf.split(X_train):
    print('Fold ',f)
    print('The training set has ', train_index.shape[0],' samples')
    print('Their index locations are: ', train_index, '\n')
    print('The validation set has ', validation_index.shape[0],' samples')
    print('Their index locations are: ', validation_index,'\n\n')
    f+=1

Fold  1
The training set has  4646  samples
Their index locations are:  [   0    1    2 ... 6190 6191 6194] 

The validation set has  1549  samples
Their index locations are:  [   5    7    8 ... 6188 6192 6193] 


Fold  2
The training set has  4646  samples
Their index locations are:  [   0    3    4 ... 6192 6193 6194] 

The validation set has  1549  samples
Their index locations are:  [   1    2    6 ... 6183 6186 6187] 


Fold  3
The training set has  4646  samples
Their index locations are:  [   1    2    4 ... 6192 6193 6194] 

The validation set has  1549  samples
Their index locations are:  [   0    3   13 ... 6185 6190 6191] 


Fold  4
The training set has  4647  samples
Their index locations are:  [   0    1    2 ... 6191 6192 6193] 

The validation set has  1548  samples
Their index locations are:  [   4    9   11 ... 6182 6189 6194] 




In [7]:
# define values to loop over
learning_rates = [0.0004, 0.0006, 0.0008, 0.001]
kernel_sizes = [(3,3),(5,5)]
batch_sizes = [256, 128, 64, 32]

# setup device to be cuda if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# choosing values with highest
topmax_avg_acc = np.zeros(32)
max_avg_stddevs = np.zeros(32)
best = np.zeros((32,3))

count = 0
for lr in learning_rates:
    for ks in kernel_sizes:
        for bs in batch_sizes:
    
            sub_accs = []
            count +=1
            print(f'Iteration {count}/{len(learning_rates)*len(kernel_sizes)*len(batch_sizes)}:')
            for train_index,validation_index in kf.split(X_train):

                # select training set using the indices found from kf.split
                X_train2, X_val = X_train[train_index], X_train[validation_index]
                # select validation set using the indices found from kf.split
                t_train2, t_val = t_train[train_index], t_train[validation_index]

                X_train2 = torch.tensor(X_train2, dtype=torch.float32).cuda()
                X_val = torch.tensor(X_val, dtype=torch.float32).cuda()
                t_train2 = torch.tensor(t_train2, dtype=torch.long).cuda()
                t_val = torch.tensor(t_val, dtype=torch.long).cuda()

                # set data loaders for train and val sets
                train_data = TensorDataset(X_train2, t_train2)
                val_data = TensorDataset(X_val, t_val)
                X_train2 = []; t_train2 = []
                loader_train = DataLoader(train_data, shuffle=True, batch_size=bs)
                loader_val = DataLoader(val_data, shuffle=True, batch_size=bs)
                train_data = []; val_data = [] # free space

                # initialize the model
                model = cnn(3,10,ks)
                # initialize our optimizer and loss function
                lossFn = nn.CrossEntropyLoss()
                optim = torch.optim.Adam(model.parameters(), lr=lr)

                # Train model
                _, _, _, val_accs = train(model,optim,lossFn,loader_train,loader_val,EPOCHS,device)
                # Performance Measure
                sub_accs.append(val_accs[len(val_accs)-1])

            avg_acc_val = np.sum(np.array(sub_accs,dtype=np.float32))/k
            val_acc_stddev = np.std(np.array(sub_accs, dtype=np.float32))
            feats = np.array([lr,ks,bs])
            #print(f'sub_accs:\n {sub_accs}\n\navg_acc_val:\n {avg_acc_val}\n\nfeats:\n {feats}')
            #print('average accuracy: ', avg_acc_val,'\n')
            #accs += [avg_acc_val]
            
            for m in range(len(topmax_avg_acc)):
                if avg_acc_val >= topmax_avg_acc[m]:
                    if m==31:
                        best = np.vstack([best, feats])[1:]
                        topmax_avg_acc = np.append(topmax_avg_acc,avg_acc_val)[1:]
                        max_avg_stddevs = np.append(max_avg_stddevs,val_acc_stddev)[1:]

                    else:
                        continue
                elif avg_acc_val < topmax_avg_acc[m]: 
                    if m == 0:
                        break
                    if m != 0:
                        topmax_avg_acc = np.append(np.append(topmax_avg_acc[:m],avg_acc_val),topmax_avg_acc[m:])[1:]
                        max_avg_stddevs = np.append(np.append(max_avg_stddevs[:m],val_acc_stddev),max_avg_stddevs[m:])[1:]
                        best = np.vstack([np.vstack([best[:m],feats]),best[m:]])[1:]
                        break
            #print(f'\ntopmax_avg_acc:\n {topmax_avg_acc}\n\nbest:\n {best}')

Iteration 1/32:


loss 0.18 acc 0.957: 100%|██████████| 100/100 [03:22<00:00,  2.03s/it]
<ipython-input-7-2254af38d13d>:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  feats = np.array([lr,ks,bs])


Iteration 2/32:


loss 0.17 acc 0.961: 100%|██████████| 100/100 [03:23<00:00,  2.04s/it]


Iteration 3/32:


...

loss 0.16 acc 0.955: 100%|██████████| 100/100 [05:15<00:00,  3.15s/it]


Iteration 31/32:


loss 0.30 acc 0.957: 100%|██████████| 100/100 [03:50<00:00,  2.31s/it]


Iteration 32/32:


loss 0.16 acc 0.967: 100%|██████████| 100/100 [03:26<00:00,  2.06s/it]


In [8]:
# print cv results
print(f'accuracy and feature rankings:\nacc, std_dev, lr, ks, bs\n\n{np.hstack([topmax_avg_acc[:,np.newaxis],max_avg_stddevs[:,np.newaxis],best])}')

accuracy and feature rankings:
acc, std_dev, lr, ks, bs

[[0.9454376697540283 0.008405357599258423 0.0006 (3, 3) 256]
 [0.9494824409484863 0.004553064703941345 0.001 (3, 3) 32]
 [0.9526563882827759 0.009018884971737862 0.0008 (3, 3) 256]
 [0.9535328149795532 0.010953939519822598 0.0004 (5, 5) 256]
 [0.954272985458374 0.004525352735072374 0.0008 (5, 5) 128]
 [0.9543731808662415 0.005717592779546976 0.001 (5, 5) 128]
 [0.9556213021278381 0.00561443530023098 0.0006 (3, 3) 128]
 [0.9557253122329712 0.003472384996712208 0.0004 (3, 3) 128]
 [0.9557651877403259 0.0029758550226688385 0.0008 (3, 3) 64]
 [0.9560989737510681 0.006363527849316597 0.001 (3, 3) 128]
 [0.9565662145614624 0.010187032632529736 0.001 (3, 3) 256]
 [0.9575688242912292 0.004712746012955904 0.0006 (3, 3) 32]
 [0.9578682780265808 0.0024163571652024984 0.0004 (3, 3) 256]
 [0.9579687714576721 0.005198695696890354 0.001 (5, 5) 64]
 [0.9584975838661194 0.009879971854388714 0.001 (3, 3) 64]
 [0.9587301015853882 0.0060800747014582